In [2]:
import pandas as pd

In [3]:
dd_bundle_subs = pd.read_csv('Bundles Subscription Behavior.csv',sep = ',')
df_data_BTL = dd_bundle_subs[dd_bundle_subs.bundle_name.str.startswith('BTL')]
df_kmeans= pd.read_csv('df_kmeans.csv',sep = ',')
df_kmeans_bundle = df_kmeans.merge(df_data_BTL,on = 'msisdn_id',how='left')

In [6]:
def reco(num):
    bundle = []
    t = df_kmeans_bundle[df_kmeans_bundle.msisdn_id==num].reset_index()
    T1 = t.groupby(['labels','bundle_name','price_point'],dropna=False)['msisdn_id'].count().nlargest(2).reset_index()

# we will perform recommendation to customer in labels 0 like this
# customer without bundle subscription will be recommend BTL 4G Bundle 1 Daily 
# customer with bundle BTL 4G Bundle 1 Daily will be recommend  BTL 4G Bundle 14 Weekly 
# customer with bundle BTL 4G Bundle 14 Weekly will be recommend BTL 4G Bundle 7 Weekly (Upselling)
    if T1.labels[0]== 0: # cluster 0
        bundle.append(T1.labels[0])
        if len(T1) == 0 :
            bundle.append('BTL 4G Bundle 1 Daily')
            bundle.append('BTL 4G Bundle 14 Weekly')
        elif len(T1) == 1 :
            if T1.bundle_name[0] == 'BTL 4G Bundle 1 Daily ':
                    bundle.append('BTL 4G Bundle 1 Daily')
                    bundle.append('BTL 4G Bundle 14 Weekly')
            elif T1.bundle_name[0] == 'BTL 4G Bundle 14 Weekly':
                    bundle.append('BTL 4G Bundle 14 Weekly')
                    bundle.append('BTL 4G Bundle 7 Weekly') # uspselling
            else:
                if T1.price_point[0] > 22 : # I want to prevent down - selling
                    bundle.append(T1.bundle_name[0])
                    bundle.append('BTL 4G Bundle 7 Weekly')
                else:
                    bundle.append('BTL 4G Bundle 14 Weekly')
                    bundle.append('BTL 4G Bundle 7 Weekly')
        else:     
            if T1.price_point[1] > 22:  # I want to prevent down - selling
                    bundle.append(T1.bundle_name[0])
                    bundle.append(T1.bundle_name[1])
            else:
                    bundle.append(T1.bundle_name[0])
                    bundle.append('BTL 4G Bundle 7 Weekly')# Upselling

# we will perform recommendation to customer in labels 1 like this
# customer without bundle subscription will be recommend BTL 4G Bundle 14 Weekly
# customer with bundle BTL 4G Bundle 14 Weekly will be recommend BTL 4G Bundle 7 Weekly 
# customer with bundle BTL 4G Bundle 7 Weekly   will be recommend BTL 4G Bundle 8 Weekly(Upselling
    
    
    if T1.labels[0]== 1:   # cluster 1
        bundle.append(T1.labels[0])
        if len(T1) == 0 :
            bundle.append('BTL 4G Bundle 7 Weekly')
            bundle.append('BTL 4G Bundle 14 Weekly')
        elif len(T1) == 1 :
            if T1.bundle_name[0] == 'BTL 4G Bundle 14 Weekly':
                    bundle.append('BTL 4G Bundle 7 Weekly')
                    bundle.append('BTL 4G Bundle 14 Weekly')
            elif T1.bundle_name[0] == 'BTL 4G Bundle 7 Weekly':
                    bundle.append('BTL 4G Bundle 7 Weekly')
                    bundle.append('BTL 4G Bundle 8 Weekly')
            else:
                if T1.price_point[0] > 35 : # I want to prevent down - selling
                    bundle.append(T1.bundle_name[0])
                    bundle.append('BTL 4G Bundle 8 Weekly')
                else:
                    bundle.append('BTL 4G Bundle 7 Weekly')
                    bundle.append('BTL 4G Bundle 8 Weekly')
        else:     
            if T1.price_point[1] > 35:  # I want to prevent down - selling
                    bundle.append(T1.bundle_name[0])
                    bundle.append(T1.bundle_name[1])
            else:
                    bundle.append(T1.bundle_name[0])
                    bundle.append('BTL 4G Bundle 8 Weekly')# Upselling

# we will perform recommendation to customer in labels 2 like this
# customer without bundle subscription will be recommend BTL 4G BTL 4G Bundle 7 Weekly  
# customer with BTL 4G Bundle 7 Weekly  will be recommend BTL 4G Bundle 8 Weekly 
# customer with bundle BTL 4G Bundle 8 Weekly  will be recommend BTL 4G Bundle 9 Monthly (Upselling)                   
     
    if T1.labels[0]== 2:   # cluster 2
        bundle.append(T1.labels[0])
        if len(T1) == 0 :
            bundle.append('BTL 4G Bundle 7 Weekly')
            bundle.append('BTL 4G Bundle 8 Weekly')
        elif len(T1) == 1 :
            if T1.bundle_name[0] == 'BTL 4G Bundle 7 Weekly':
                    bundle.append('BTL 4G Bundle 7 Weekly')
                    bundle.append('BTL 4G Bundle 8 Weekly')
            elif T1.bundle_name[0] == 'BTL 4G Bundle 8 Weekly':
                    bundle.append('BTL 4G Bundle 8 Weekly')
                    bundle.append('BTL 4G Bundle 9 Weekly')
            else:
                if T1.price_point[0] > 50 : # I want to prevent down - selling
                    bundle.append(T1.bundle_name[0])
                    bundle.append('BTL 4G Bundle 9 Weekly')
                else:
                    bundle.append('BTL 4G Bundle 8 Weekly')
                    bundle.append('BTL 4G Bundle 9 Weekly')
        else:     
            if T1.price_point[1] > 50:  # I want to prevent down - selling
                    bundle.append(T1.bundle_name[0])
                    bundle.append(T1.bundle_name[1])
            else:
                    bundle.append(T1.bundle_name[0])
                    bundle.append('BTL 4G Bundle 8 Weekly')# Upselling
                    
    return bundle

    
 




In [18]:
num = 36
t = df_kmeans_bundle[df_kmeans_bundle.msisdn_id==num].head().reset_index()
T1 = t.groupby(['labels','bundle_name','price_point'],dropna=False)['msisdn_id'].count().nlargest(2).reset_index()
T1

,labels,bundle_name,price_point,msisdn_id
0,1,BTL 4G Bundle 9 Monthly,72.727273,4
1,1,BTL 4G Bundle 8 Weekly,50.909091,1


In [19]:
# reco result are segment,bundle 1, bundle 2
reco(36)

[1, 'BTL 4G Bundle 9 Monthly', 'BTL 4G Bundle 8 Weekly']